In [1]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [2]:
# Starting from Python 3.8 DLL search policy has changed.
# We need to add path to CUDA DLLs explicitly.
import os

if os.name == "nt":
    # Add CUDA_PATH env variable
    cuda_path = os.environ["CUDA_PATH"]
    if cuda_path:
        os.add_dll_directory(os.path.join(cuda_path, "bin"))
    else:
        raise StopExecution

In [3]:
from PIL import Image
from IPython.display import display
from io import BytesIO

import python_vali as vali
import numpy as np

from ipywidgets import interact
import ipywidgets as widgets

In [4]:
## generate test video. not secured as it uses os.system
## lines in ffmpegcmd may be commented out using '#' char
import tempfile
ffmpegcmd='''ffmpeg -f lavfi 
-i testsrc=duration=300:size=640x480:rate=30 
-filter_complex "drawtext=fontsize=42: text='%{pts \:flt}  - %{frame_num}': x=100 : y=50 : box=1" 
-profile:v baseline 
-pix_fmt yuv420p 
-g 120 -keyint_min 120 
-y
'''
ffmpegcmd=' '.join([s for s in ffmpegcmd.split('\n') if not s.startswith("#")])
tmpfile=tempfile.gettempdir()+"/output.mp4"
if not os.path.isfile(tmpfile):
    os.system(ffmpegcmd+" "+tempfile.gettempdir()+"/output.mp4")

In [5]:
url = "/tmp/output.mp4"

In [6]:
# GPU-accelerated decoder
pyDec = vali.PyDecoder(
    url,
    {},
    gpu_id=0)

# GPU-accelerated converter
pyCvt = vali.PySurfaceConverter(
    pyDec.Format,
    vali.PixelFormat.RGB,
    gpu_id=0)

# GPU-accelerated JPEG encoder.
# It's faster to encode Surface on GPU and show JPEG in widget.
pyJpeg = vali.PyNvJpegEncoder(gpu_id=0)
pyJpegEncCtx = pyJpeg.Context(
    compression=100,
    pixel_format=vali.PixelFormat.RGB)

In [16]:
#@interact(seek_frame=widgets.IntSlider(
#    min=0, max=pyDec.NumFrames - 1, step=1, value=0))
def decode_show_video_frame(seek_frame: int, keyframe: bool) -> None:
    # Raw decoded Surface
    surf_src = vali.Surface.Make(
        format=pyDec.Format,
        width=pyDec.Width,
        height=pyDec.Height,
        gpu_id=0)

    # Raw Surface, converted to RGB
    surf_dst = vali.Surface.Make(
        format=vali.PixelFormat.RGB,
        width=pyDec.Width,
        height=pyDec.Height,
        gpu_id=0)

    seek_ctx = vali.SeekContext(seek_frame,
                                mode=vali.SeekMode.EXACT_FRAME if keyframe else vali.SeekMode.PREV_KEY_FRAME)

    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src, seek_ctx)
    if not success:
        print(details)
        raise StopExecution

    # Convert tot RGB
    success, details = pyCvt.Run(surf_src, surf_dst)
    if not success:
        print(details)
        raise StopExecution

    # Compress to JPEG
    buffers, details = pyJpeg.Run(pyJpegEncCtx, [surf_dst])
    if len(buffers) != 1:
        print(details)
        raise StopExecution

    # Display in notebook.
    # The same picture is shown twice for some reason.
    jpeg_bytes = BytesIO(np.ndarray.tobytes(buffers[0]))
    display(Image.open(jpeg_bytes), display_id="decoded_frame")

In [17]:
## by the way, pyDec.Duration should be 300.00 if using the generated videos!
## to be solved in another PR!
print(f"Reported duration: {pyDec.Duration} s (expected 300.00)")
print(f"Computed duration: {pyDec.NumFrames/pyDec.Framerate} s (expected 300.00)")
print(f"Gop size: {pyDec.GopSize} (expected 120)")
print(f"NumFrame: {pyDec.NumFrames} (expected 9000)")
print(f"Framerate: {pyDec.Framerate} (expected 30.00)")

Reported duration: 4.608 s (expected 300.00)
Computed duration: 300.0 s (expected 300.00)
Gop size: 12 (expected 120)
NumFrame: 9000 (expected 9000)
Framerate: 30.0 (expected 30.00)


In [18]:
interact(decode_show_video_frame,
         #seek_frame=widgets.IntSlider(min=0, max=pyDec.NumFrames - 1, step=1, value=0)
         seek_frame=widgets.FloatSlider(min=0, max=pyDec.NumFrames/pyDec.Framerate, value=0)
         keyframe=False)

interactive(children=(FloatSlider(value=0.0, description='seek_frame', max=300.0), Output()), _dom_classes=('w…

<function __main__.decode_show_video_frame(seek_frame: int) -> None>